In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./Datasets/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./Datasets/dev_data\dev_filtered.csv
./Datasets/dev_data\dev_rehydrated.jsonl
./Datasets/dev_data\jsonl2csv.ipynb
./Datasets/dev_data\.ipynb_checkpoints\dev_filtered-checkpoint.csv
./Datasets/dev_data\.ipynb_checkpoints\dev_rehydrated-checkpoint.jsonl
./Datasets/dev_data\.ipynb_checkpoints\jsonl2csv-checkpoint.ipynb
./Datasets/test_data\jsonl2csv.ipynb
./Datasets/test_data\test_filtered.csv
./Datasets/test_data\test_rehydrated.jsonl
./Datasets/test_data\.ipynb_checkpoints\jsonl2csv-checkpoint.ipynb
./Datasets/test_data\.ipynb_checkpoints\test_filtered-checkpoint.csv
./Datasets/test_data\.ipynb_checkpoints\test_rehydrated-checkpoint.jsonl
./Datasets/train_data\generated_data_conspiracy.csv
./Datasets/train_data\jsonl2csv.ipynb
./Datasets/train_data\train_conspiracy.csv
./Datasets/train_data\train_filtered.csv
./Datasets/train_data\train_rehydrated.jsonl
./Datasets/train_data\.ipynb_checkpoints\generated_data_conspiracy-checkpoint.csv
./Datasets/train_data\.ipynb_checkpoints\jsonl2csv-ch

安装依赖包

In [2]:
# !pip install tweet-preprocessor

In [3]:
import pandas as pd
import re, string
import numpy as np

import preprocessor as p

In [4]:
import random, torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [5]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [7]:
path_train = "./Datasets/train_data/train_filtered.csv"
path_val = "./Datasets/train_data/train_filtered.csv" 
path_test = "./Datasets/dev_data/dev_filtered.csv"
path_test_final = "./Datasets/test_data/test_filtered.csv"

# Read dataset

In [8]:
import string,re
def preprocessing_text(text):
    text = p.clean(text) # This method change the preprocessing function from simple version to enhanced version
    
    text = text.strip()
    text = text.translate(text.maketrans('', '', string.punctuation.replace("_","")))
    text = re.sub('\\s+',' ',text).strip()
    return text

In [9]:
import pandas as pd

def convert_label(text):
    if text == "Yes":
        return 1
    else:
        return 0

def read_and_preprocessing(path_data):
    df = pd.read_csv(path_data)
    df["category"] = df["conspiracy"].apply(convert_label)
    x_input = df["text"].apply(preprocessing_text).tolist()
    y_output = df["category"].tolist()
    ids = df["_id"].tolist()
    return x_input,y_output,ids

train_texts, train_labels, train_ids = read_and_preprocessing(path_train)
valid_texts, valid_labels, valid_ids = read_and_preprocessing(path_val)
print(len(train_texts),len(train_labels))
print(len(valid_texts),len(valid_labels))

3531 3531
3531 3531


In [10]:
print(preprocessing_text(train_texts[0]))

A great article on whats taking place in Bolivia referencing some similar US backed coups in the region as well as recounting some of Bolivias history and western policy towards the country


In [11]:
df_train = pd.DataFrame(list(zip(train_texts, train_labels)),
               columns =['x_data', 'y_output'])
df_train.head()

,x_data,y_output
0,A great article on whats taking place in Boliv...,1
1,Germany has upset other EU member states by se...,0
2,The cruise ship has been stuck at sea with cor...,0
3,The Jesuits were expelled from Rome in the ord...,0
4,So they want us to believe it was a suicide wh...,1


In [12]:
df_train.head(-1)

,x_data,y_output
0,A great article on whats taking place in Boliv...,1
1,Germany has upset other EU member states by se...,0
2,The cruise ship has been stuck at sea with cor...,0
3,The Jesuits were expelled from Rome in the ord...,0
4,So they want us to believe it was a suicide wh...,1
...,...,...
3525,Between the Pentagon having a secret army of r...,1
3526,Ill never understand why Feminists say its bad...,0
3527,Biden crime family at it again Caroline Biden ...,0
3528,Gab Comment text ReclaimTheNetorg There are pl...,1


# Build Filter Classifier

In [13]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Try different models here:
model_name = "models--microsoft--deberta-v3-base" 
model_name2 = ""
model_name3 = ""

bert_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)

C:\Users\8888\Anaconda3\envs\pythonProject8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at models--microsoft--deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
max_length = 512
train_encodings = tokenizer(train_texts, truncation=True, max_length=max_length, padding=True)

In [15]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)

In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,             # total number of training epochs
    learning_rate=2e-5,
    per_device_train_batch_size=32,  # batch size per device during training
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    save_total_limit = 1,
    report_to = "tensorboard"
)

trainer = Trainer(
    model=bert_model,                    # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)
trainer.train()
# API: fac85ccacc3dffb183116aba932e6bcc08010443

Step,Training Loss
100,0.675800
200,0.544700
300,0.434100
400,0.337500
500,0.298500
600,0.223200
700,0.163700
800,0.137500
900,0.109900
1000,0.102600


TrainOutput(global_step=1110, training_loss=0.28165548653215977, metrics={'train_runtime': 20301.8949, 'train_samples_per_second': 1.739, 'train_steps_per_second': 0.055, 'total_flos': 3973916674288440.0, 'train_loss': 0.28165548653215977, 'epoch': 10.0})

# Inference

In [17]:
def make_prediction(review,tokenizer,trainer):
    demo_input = preprocessing_text(review)
    demo_encodings = tokenizer([demo_input], truncation = True, max_length = max_length, padding = True)
    test_dataset = CustomDataset(demo_encodings, [0])
    predic_demo = trainer.predict(test_dataset)[0]
    predict_label = np.argmax(predic_demo, axis=1).flatten().tolist()[0]
    return predict_label

In [18]:
y_pred = []

for review in valid_texts:
    a = make_prediction(review,tokenizer,trainer)
    y_pred.append(a)

In [19]:
from sklearn.metrics import *

print("M_Pr: ", round(precision_score(valid_labels, y_pred, average='macro'),4))
print("M_Re: ", round(recall_score(valid_labels, y_pred, average='macro'),4))
print("M_F1: ", round(f1_score(valid_labels, y_pred, average='macro'),4))

print("W_Pr: ", round(precision_score(valid_labels, y_pred, average='weighted'),4))
print("W_Re: ", round(recall_score(valid_labels, y_pred, average='weighted'),4))
print("W_F1: ", round(f1_score(valid_labels, y_pred, average='weighted'),4))

print("acc:", round(accuracy_score(valid_labels, y_pred), 4))

M_Pr:  0.9735
M_Re:  0.9729
M_F1:  0.9732
W_Pr:  0.9737
W_Re:  0.9737
W_F1:  0.9737
acc: 0.9737


In [20]:
# Calculate precision, recall, and F1-score for different averaging methods
M_Pr = round(precision_score(valid_labels, y_pred, average='macro'), 4)
M_Re = round(recall_score(valid_labels, y_pred, average='macro'), 4)
M_F1 = round(f1_score(valid_labels, y_pred, average='macro'), 4)

W_Pr = round(precision_score(valid_labels, y_pred, average='weighted'), 4)
W_Re = round(recall_score(valid_labels, y_pred, average='weighted'), 4)
W_F1 = round(f1_score(valid_labels, y_pred, average='weighted'), 4)

acc = round(accuracy_score(valid_labels, y_pred), 4)

# Create a string with the formatted output
output_string = f"M_Pr: {M_Pr}\n" \
                f"M_Re: {M_Re}\n" \
                f"M_F1: {M_F1}\n" \
                f"W_Pr: {W_Pr}\n" \
                f"W_Re: {W_Re}\n" \
                f"W_F1: {W_F1}\n" \
                f"acc: {acc}\n"

# Save the output to a text file named "my_scores_eng.txt"
file_path = "Task2_submission_validation/my_scores_eng.txt"
with open(file_path, "w") as f:
    f.write(output_string)

# Submission for validation datasets

In [21]:
def convert2category(y_pred):
    y_label = []
    for y in y_pred:
        if y == 0:
              y_label.append("No")
        else:
              y_label.append("Yes")
    return y_label

In [22]:
name_sub = "Task2_submission_validation/submission.csv"
y_pred_label = convert2category(y_pred)
df_sub = pd.DataFrame(list(zip(valid_ids, y_pred_label)),
               columns =['_id', 'conspiracy'])
df_sub.to_csv(name_sub, index=False)
df_sub.head()

,_id,conspiracy
0,t1_f7ju17o,Yes
1,t1_givys64,No
2,t1_fmcunuq,No
3,t1_jolyktt,No
4,t1_exd9ccm,Yes


# Submission for test datasets


In [23]:
def read_and_preprocessing_for_test(path_data):
    df = pd.read_csv(path_data)
    x_input = df["text"].apply(preprocessing_text).tolist()
    ids = df["_id"].tolist()
    return x_input,ids

test_texts,test_ids = read_and_preprocessing_for_test(path_test)

def make_prediction(review,tokenizer,trainer):
    demo_input = preprocessing_text(review)
    demo_encodings = tokenizer([demo_input], truncation = True, max_length = max_length, padding = True)
    test_dataset = CustomDataset(demo_encodings, [0])
    predic_demo = trainer.predict(test_dataset)[0]
    predict_label = np.argmax(predic_demo, axis=1).flatten().tolist()[0]
    return predict_label


In [24]:
y_pred_test = []

for review in test_texts:
    a = make_prediction(review,tokenizer,trainer)
    y_pred_test.append(a)

In [25]:
def convert2category(y_pred):
    y_label = []
    for y in y_pred:
        if y == 0:
              y_label.append("No")
        else:
              y_label.append("Yes")
    return y_label

In [26]:
name_sub = "Task2_submission_development/submission.csv"
y_pred_label = convert2category(y_pred_test)
df_sub = pd.DataFrame(list(zip(test_ids, y_pred_label)),
               columns =['_id', 'conspiracy'])
df_sub.to_csv(name_sub, index=False)
df_sub.head()

,_id,conspiracy
0,t1_emz6exn,Yes
1,t1_f07ejkp,Yes
2,t1_f6mnlr1,Yes
3,t1_iiv2878,No
4,t1_gs39jid,No


# Submission for test_final datasets


In [27]:
def read_and_preprocessing_for_test(path_data):
    df = pd.read_csv(path_data)
    x_input = df["text"].apply(preprocessing_text).tolist()
    ids = df["_id"].tolist()
    return x_input,ids

test_texts,test_ids = read_and_preprocessing_for_test(path_test_final)

def make_prediction(review,tokenizer,trainer):
    demo_input = preprocessing_text(review)
    demo_encodings = tokenizer([demo_input], truncation = True, max_length = max_length, padding = True)
    test_dataset = CustomDataset(demo_encodings, [0])
    predic_demo = trainer.predict(test_dataset)[0]
    predict_label = np.argmax(predic_demo, axis=1).flatten().tolist()[0]
    return predict_label


In [28]:
y_pred_test = []

for review in test_texts:
    a = make_prediction(review,tokenizer,trainer)
    y_pred_test.append(a)

In [29]:
def convert2category(y_pred):
    y_label = []
    for y in y_pred:
        if y == 0:
              y_label.append("No")
        else:
              y_label.append("Yes")
    return y_label

In [30]:
name_sub = "Task2_submission_test/submission.csv"
y_pred_label = convert2category(y_pred_test)
df_sub = pd.DataFrame(list(zip(test_ids, y_pred_label)),
               columns =['_id', 'conspiracy'])
df_sub.to_csv(name_sub, index=False)
df_sub.head()

,_id,conspiracy
0,t1_c7tn9ef,No
1,t1_cb06xx0,No
2,t1_ch7vf80,No
3,t1_ch95xdq,No
4,t1_chfo9q8,No
